In [3]:
import telebot
from telebot import types
import random

In [ ]:
bot = telebot.TeleBot('###')
 
markup = types.ReplyKeyboardMarkup()
btn_1 = types.KeyboardButton('Расскажи что-нибудь интересное о финансах')
btn_2 = types.KeyboardButton('Рассчитай приведённую доходность инвестиции')
markup.add(btn_1, btn_2)

fun_key = types.ReplyKeyboardMarkup() 
btn_1 = types.KeyboardButton('Анекдот')
btn_2 = types.KeyboardButton('Полезная информация')
btn_3 = types.KeyboardButton('В главное меню')
fun_key.row(btn_1, btn_2)
fun_key.row(btn_3)

empt = types.ReplyKeyboardRemove()

users = {} #словарь, ключи = id пользователей, значения = словарь с ключами параметрами и значениями значениями 

aneki = ['Встречаются два студента ВШЭ и один другому: "Слышал, ты сдал экзамен по международным финансам?"\
 - "Да, и теперь я знаю, что международные финансы не международны."', "Да что же это опять-то за пробой фрактала по Демарку через Фибоначчи!!!\
 – подумал трейдер и грязно выругался.", 'Заходит мужик в банк. Берёт вклад. А он ему как раз', "Какой смысл в том, чтобы хранить деньги в банке?\
 Ведь лучше всего они хранятся в кошельке — там они точно не могут 'испариться'.", 'Как говорит наш финансовый директор:\
 "Херовая маржа - это все-таки лучше, чем моржовый хер".', 'Когда у мужчины есть деньги - это наш общий финансовый достаток,\
 а когда у мужчины нет денег - это его личный финансовый недостаток.', 'Никто не умеет прощать так, как гаишники за деньги','После кофе\
 в слое от финансовой подушки безопасности осталась лишь наволочка', 'Любой кредит подобен  клизме, вам делают финансовое вливание\
 с целью в дальнейшем полностью опорожнить ваш кошелёк', 'А ведь я был практически здоров, пока врач не узнал, что я финансовый директор банка' ]

no_homo = lambda x: x.replace(',', '.') if ',' in x else x

@bot.message_handler(commands = ['start'])
def start(message):  
    bot.send_message(message.chat.id, f'Здравствуйте, {message.from_user.first_name}\n\nМеня зовут Филипп, я персональный финансовый ассистент\n\nПодскажите,\
 как я могу помочь вам сегодня?', reply_markup = markup)

@bot.message_handler(regexp = 'Расскажи что-нибудь интересное о финансах')
def fun(message): 
    bot.send_message(message.chat.id, 'Что конкретно вы хотите услышать?', reply_markup = fun_key)
    
@bot.message_handler(regexp = 'Анекдот')
def anekdot(message):
    bot.send_message(message.chat.id, f'{random.choice(aneki)}', reply_markup = fun_key )
    
@bot.message_handler(regexp = 'Полезная информация')
def info(message):
    articles = types.InlineKeyboardMarkup()
    btn_1 = types.InlineKeyboardButton('Что такое центральный банк?', url = 'https://finlit.uz/ru/articles/financial-market/central-bank/' )
    btn_2 = types.InlineKeyboardButton('Коммерческие банки', url = 'https://finlit.uz/ru/articles/financial-market/commercial-banks/' )
    btn_3 = types.InlineKeyboardButton('Наличные и безналичные деньги', url = 'https://finlit.uz/ru/articles/money/cash-and-non-cash-money/' )
    btn_4 = types.InlineKeyboardButton('Валюта', url = 'https://finlit.uz/ru/articles/money/currencies/' )
    articles.row(btn_1, btn_2)
    articles.row(btn_3, btn_4)
    
    bot.send_message(message.chat.id, 'Веберите интересующую тему:', reply_markup = articles )
    
@bot.message_handler(regexp = 'В главное меню')
def main_menu(message):
    bot.send_message(message.chat.id, 'Как я могу помочь вам сегодня?', reply_markup = markup)
    
@bot.message_handler(regexp = 'Рассчитай приведённую доходность инвестиции')
def message_reply(message):
    bot.send_message(message.chat.id, 'Сколько планируете инвестировать?', reply_markup = empt)
    bot.register_next_step_handler(message, on_click)
    
def on_click(message):
    sum_ = float((no_homo(message.text.strip('руб₽йцкенгшщзхъфывапролджэячсмить!"№;%:?*()_ю'))).replace(' ', ''))
    bot.send_message(message.chat.id, 'Под какую годовую доходность?\n\n<em>Введите число в процентах, к примеру: 18</em>', parse_mode = "html")
    bot.register_next_step_handler(message, on_click_2)     
    users[message.from_user.id] = {'sum_' : sum_} 
    
def on_click_2(message):
    ret = float((no_homo(message.text.strip('руб₽йцкенгшщзхъфывапролджэячсмить!"№;%:?*()_ю'))).replace(' ', ''))
    bot.send_message(message.chat.id, 'На сколько месяцев вы рассчитываете инвестировать?')
    bot.register_next_step_handler(message, on_click_3)  
    users[message.from_user.id]['ret'] = ret 
    
def on_click_3(message):
    time = float(message.text.strip('руб₽йцкенгшщзхъфывапролджэячсмить!"№;%:?*()_ю'))
    bot.send_message(message.chat.id, 'Какую годовую инфляцию вы ожидаете?\n\n<em>Введите число в процентах, к примеру: 16</em>', parse_mode = "html")
    bot.register_next_step_handler(message, on_click_4) 
    users[message.from_user.id]['time'] = time 
    
def on_click_4(message):
    infl = float((no_homo(message.text.strip('руб₽йцкенгшщзхъфывапролджэячсмить!"№;%:?*()_ю'))).replace(' ', ''))
    users[message.from_user.id]['infl'] = infl 
    inv_yield = users[message.from_user.id]['sum_']*((1 + (users[message.from_user.id]['ret']/100))/(1 + (users[message.from_user.id]['infl']/100)))**(users[message.from_user.id]['time']/12)
    real_ret = (inv_yield/users[message.from_user.id]['sum_']) - 1
    bot.send_message(message.chat.id, f'Приведённая стоимость инвестии составляет: {round(inv_yield, 2)}₽\nРеальная доходность: {round((real_ret * 100) , 2)}%', reply_markup = markup)  
bot.polling(none_stop = True)